In [87]:
import georinex as gr
import numpy as np
import json

In [88]:
# Указываем путь к файлу SP3
sp3_file_path = '../data/GSR_1-3D_24011812.sp3'

In [96]:
# Загружаем данные из файла SP3
obs = gr.load(sp3_file_path, use='G').dropna(dim='time', how='all')
obs

<xarray.Dataset>
Dimensions:   (time: 288, sv: 119, ECEF: 3)
Coordinates:
  * time      (time) datetime64[ns] 2024-01-18T12:00:00 ... 2024-01-21T11:45:00
  * sv        (sv) <U3 'C06' 'C07' 'C08' 'C09' 'C10' ... 'G32' 'J02' 'J03' 'J04'
  * ECEF      (ECEF) <U1 'x' 'y' 'z'
Data variables:
    position  (time, sv, ECEF) float64 -2.143e+03 3.121e+04 ... -7.083e+03
    clock     (time, sv) float64 55.33 -657.1 104.0 ... -1.335 -0.1851 77.57
    velocity  (time, sv, ECEF) float64 0.0 6.046e-154 ... 2.991e-67 2.743e-57
    dclock    (time, sv) float64 7.145e-67 6.047e-154 ... 3.331e+35 3.125e-33
Attributes:
    t0:          2024-01-18 12:00:00
    Nepoch:      288
    coord_sys:   IGS20
    orbit_type:  EXT
    agency:       GSR

In [90]:
times_obs = obs.time.data

In [91]:
dct_to_save = {}
for t in times_obs:
    obs_pd_temp = obs.sel(time=t).to_dataframe().reset_index().dropna()
    obs_pd_temp = obs_pd_temp.groupby('sv').agg(list).reset_index()
    del obs_pd_temp['time']
    del obs_pd_temp['ECEF']
    obs_pd_temp['clock'] = obs_pd_temp['clock'].apply(lambda x: x[0])
    obs_pd_temp['dclock'] = obs_pd_temp['dclock'].apply(lambda x: x[0])
    obs_pd_temp = {str(t): obs_pd_temp.to_dict('list')}
    dct_to_save.update(obs_pd_temp)

In [109]:
with open('../data/satellite_data.json', 'w') as f:
    json.dump(dct_to_save, f)
